<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/ArabicPIIRedaction/blob/main/fine-tuning_SLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [7]:
import pandas as pd

## Testing data
- Original Competition Data

In [8]:
data_url = "https://raw.githubusercontent.com/MuhammadHelmyOmar/ArabicPIIRedaction/main/data/data_final.xlsx"

test = pd.read_excel(data_url)

test.head()

,source,target,dialect
0,الأستاذ أدولفوس ريغان زييمان، بصفتك المدير الت...,الأستاذ [MASK]، بصفتك المدير التنفيذي الرئيسي ...,EGYPT
1,مرحبًا يا هانا، ممكن لو سمحتي تتحققي من التداع...,مرحبًا يا [MASK]، ممكن لو سمحتي تتحققي من التد...,EGYPT
2,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,SAUDI
3,عزيزي ديفان، مطلوب تقديم عرض تقديمي على مستوى ...,عزيزي [MASK]، مطلوب تقديم عرض تقديمي على مستوى...,EGYPT
4,ممكن كمان نعمل جلسة عن كيفية إدارة التوتر والت...,The sentence doesn't contain any personal or s...,LEVANTINE


## Training data

In [9]:
from google.colab import drive
import random

### Loading Data

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# All Data

data_path = "/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/ALL_DATA.csv"

train = pd.read_csv(data_path)

In [12]:
print(len(train))
train.head()

328597


,source,dialect,tokens,tags,clean_source
0,انضم لينا في المعرض العلمي عند الإحداثيات <!NE...,eg,"['انضم', 'لينا', 'في', 'المعرض', 'العلمي', 'عن...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEARBYG...",انضم لينا في المعرض العلمي عند الإحداثيات [-18...
1,انضم لمجتمعنا أونلاين للصحة النفسية والرفاهية....,eg,"['انضم', 'لمجتمعنا', 'أونلاين', 'للصحة', 'النف...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",انضم لمجتمعنا أونلاين للصحة النفسية والرفاهية....
2,<!FIRSTNAME_1!> لورينزا </!FIRSTNAME_1!>، وجود...,eg,"['لورينزا', '،', 'وجودك', 'في', 'الندوة', '""',...","['B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', ...","لورينزا، وجودك في الندوة ""اختيارات الغذاء الصح..."
3,شركة ناشئة في مجال الحلول بتدور على توظيف <!JO...,eg,"['شركة', 'ناشئة', 'في', 'مجال', 'الحلول', 'بتد...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-JO...",شركة ناشئة في مجال الحلول بتدور على توظيف منسق...
4,عزيزي <!FIRSTNAME_1!> جيف </!FIRSTNAME_1!>، اح...,eg,"['عزيزي', 'جيف', '،', 'احنا', 'لقينا', 'نشاط',...","['O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', ...",عزيزي جيف، احنا لقينا نشاط مشبوه ممكن يكون حصل...


In [13]:
print(train['dialect'].unique())
print(train['dialect'].value_counts())

['eg' 'en' 'levantine' 'msa' 'saudi' 'LEVANTINE' '0']
dialect
msa          199364
levantine     33296
eg            33021
saudi         32948
en            20000
LEVANTINE      9967
0                 1
Name: count, dtype: int64


In [14]:
train.loc[train['dialect']=='0', 'dialect'] = 'eg'
print(train['dialect'].value_counts())

dialect
msa          199364
levantine     33296
eg            33022
saudi         32948
en            20000
LEVANTINE      9967
Name: count, dtype: int64




---



In [15]:
# Data augmented with Arabic names and locations

names_loc_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/names_locations_augmented_data.csv")

print(len(names_loc_data))
names_loc_data.head()

47677


,source,dialect,tokens,tags,clean_source
0,رح نحط قضاة عند الجناح الجنوبي الشرقي من مكان ...,levantine,"['رح', 'نحط', 'قضاة', 'عند', 'الجناح', 'الجنوب...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",رح نحط قضاة عند الجناح الجنوبي الشرقي من مكان ...
1,مساء الخير، خلال اجتماعنا السنوي في <!CITY_1!>...,msa,"['مساء', 'الخير', '،', 'خلال', 'اجتماعنا', 'ال...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CITY', ...",مساء الخير، خلال اجتماعنا السنوي في منطقة الاو...
2,بالنسبة للمخزون في فرع <!ZIPCODE_1!> 52884 </!...,LEVANTINE,"['بالنسبة', 'للمخزون', 'في', 'فرع', '52884', '...","['O', 'O', 'O', 'O', 'B-ZIPCODE', 'O', 'O', 'O...",بالنسبة للمخزون في فرع 52884، معدات العناية ال...
3,يرجى التأكد من أن سجل المريض الأول كامل وجاهز ...,saudi,"['يرجى', 'التأكد', 'من', 'أن', 'سجل', 'المريض'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",يرجى التأكد من أن سجل المريض الأول كامل وجاهز ...
4,هذا إشعار لـ <!FIRSTNAME_1!> آية </!FIRSTNAME_...,msa,"['هذا', 'إشعار', 'لـ', 'آية', 'عضل', '.', 'يمك...","['O', 'O', 'O', 'B-FIRSTNAME', 'B-LASTNAME', '...",هذا إشعار لـ آية عضل. يمكن تحويل المبلغ المستح...


In [16]:
train = pd.concat([train, names_loc_data], ignore_index=True)
print(len(train))
train['dialect'].value_counts()

376274


,count
dialect,
msa,208928
levantine,42830
eg,42592
saudi,42458
en,20000
LEVANTINE,19466


In [17]:
# Shuffle the data

train = train.sample(frac=1).reset_index(drop=True)

### Cleaning and preprocessing

In [18]:
idx = random.randint(0, len(train))
idx

362960

In [19]:
print(train.tokens[idx])
print(type(train.tokens[idx]))
print(train.tags[idx])
print(type(train.tags[idx]))
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

['إيرين', '،', 'بصفتك', 'ممثل', 'الأمن', 'العالمي', 'في', 'الاتصالات', '،', 'يرجى', 'مشاركة', 'خططك', 'الاستراتيجية', '.', 'أرسل', 'التفاصيل', 'إلى', 'Jacky', '.', 'Dare70', '@', 'hotmail', '.', 'com', '.']
<class 'str'>
['B-FIRSTNAME', 'O', 'O', 'B-JOBTITLE', 'I-JOBTITLE', 'I-JOBTITLE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'O']
<class 'str'>
206
204


In [20]:
import re
import ast

In [21]:
tokens_list = ast.literal_eval(train.tokens[idx])
print(tokens_list)
print(type(tokens_list), len(tokens_list))

tags_list = ast.literal_eval(train.tags[idx])
print(tags_list)
print(type(tags_list), len(tags_list))

['إيرين', '،', 'بصفتك', 'ممثل', 'الأمن', 'العالمي', 'في', 'الاتصالات', '،', 'يرجى', 'مشاركة', 'خططك', 'الاستراتيجية', '.', 'أرسل', 'التفاصيل', 'إلى', 'Jacky', '.', 'Dare70', '@', 'hotmail', '.', 'com', '.']
<class 'list'> 25
['B-FIRSTNAME', 'O', 'O', 'B-JOBTITLE', 'I-JOBTITLE', 'I-JOBTITLE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'O']
<class 'list'> 25


In [22]:
# Convert tokens and tags columns to lists

train['tokens'] = train["tokens"].apply(ast.literal_eval)
train['tags'] = train['tags'].apply(ast.literal_eval)

In [23]:
print(train.tokens[idx])
print(type(train.tokens[idx]))
print(train.tags[idx])
print(type(train.tags[idx]))
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

['إيرين', '،', 'بصفتك', 'ممثل', 'الأمن', 'العالمي', 'في', 'الاتصالات', '،', 'يرجى', 'مشاركة', 'خططك', 'الاستراتيجية', '.', 'أرسل', 'التفاصيل', 'إلى', 'Jacky', '.', 'Dare70', '@', 'hotmail', '.', 'com', '.']
<class 'list'>
['B-FIRSTNAME', 'O', 'O', 'B-JOBTITLE', 'I-JOBTITLE', 'I-JOBTITLE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'I-EMAIL', 'O']
<class 'list'>
25
25


In [24]:
# Checking if there is mismatching lengths between tokens and tags

length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")


Number of rows with mismatching lengths: 0


In [25]:
tags_to_mask = ['PHONEIMEI',
 'VEHICLEVRM',
 'LITECOINADDRESS',
 'CREDITCARDNUMBER',
 'DATE',
 'NEARBYGPSCOORDINATE',
 'BITCOINADDRESS',
 'GENDER',
 'PERSONNAME',
 'JOBTITLE',
 'TIME',
 'CURRENCY',
 'BIC',
 'MASKEDNUMBER',
 'STREET',
 'MAC',
 'DOB',
 'SECONDARYADDRESS',
 'CREDITCARDISSUER',
 'ZIPCODE',
 'USERAGENT',
 'CURRENCYSYMBOL',
 'JOBTYPE',
 'BUILDINGNUMBER',
 'AGE',
 'MIDDLENAME',
 'CREDITCARDINUMBER',
 'ACCOUNTNUMBER',
 'PIN',
 'FIRSTNAME',
 'ORDINALDIRECTION',
 'PASSWORD',
 'PHONENUMBER',
 'IPV4',
 'CREDITCARDCVV',
 'USERNAME',
 'HEIGHT',
 'CURRENCYCODE',
 'ACCOUNTNAME',
 'IBAN',
 'AMOUNT',
 'PREFIX',
 'VEHICLEVIN',
 'SEX',
 'EMAIL',
 'ETHEREUMADDRESS',
 'IPV6',
 'SSN',
 'URL',
 'LASTNAME',
 'CURRENCYNAME',
 'IP']

In [26]:
def masker(row, tags_to_mask, discrepancy_list):
    """
    Constructs a masked sentence and records discrepancies, attempting to preserve original formatting.

    Args:
    row: A pandas DataFrame row with 'tokens', 'tags', and 'clean_source' columns.
    tags_to_mask: List of tag entity types to mask.
    discrepancy_list: A list to append discrepancy details.

    Returns:
    The constructed sentence with masked tokens for the given row, attempting to preserve spacing.
    """
    tokens = row['tokens']
    tags = row['tags']
    clean_source = row['clean_source']
    masked_sentence = ""
    current_position = 0

    for i in range(len(tokens)):
        token = tokens[i]
        tag = tags[i]

        # Find the position of the current token in the original string starting from the last processed position
        start_index = clean_source.find(token, current_position)

        if start_index != -1:
            # Append the text from the current position up to the start of the token
            masked_sentence += clean_source[current_position:start_index]

            if tag[2:] in tags_to_mask:
                masked_sentence += '[MASK]'
            else:
                masked_sentence += token

            # Update the current position to the end of the current token
            current_position = start_index + len(token)
        else:
            # If token not found at or after current_position, it indicates a discrepancy
            print(f"Warning: Token '{token}' not found in clean_source at or after position {current_position}\n")

            # Record the discrepancy
            discrepancy_list.append({
                'clean_source': clean_source,
                'tokens': tokens,
                'tags': tags,
                'discrepancy_token': token,
                'discrepancy_position': current_position
            })

            # You might choose how to handle the token that wasn't found -
            # here, we'll just append it unmasked to keep the process going
            masked_sentence += token
            # We don't update current_position based on this token as it wasn't found correctly

    # Append any remaining text after the last token
    masked_sentence += clean_source[current_position:]
    return masked_sentence

In [27]:
# Initialize a list to store discrepancy information
discrepancy_data = []

# Creating a new masked sentence
train['masked_source'] = train.apply(lambda row: masker(row, tags_to_mask, discrepancy_data), axis=1)

# Convert the list of discrepancy data into a DataFrame
discrepancy_df = pd.DataFrame(discrepancy_data)

In [28]:
print(f"{len(discrepancy_df)} discrepancies are found")

107 discrepancies are found


In [29]:
print(f"{len(discrepancy_df.clean_source.unique())} sentences are detected as discrepancies.")

7 sentences are detected as discrepancies.


In [30]:
# Drop discrepancies

print(f"Original number of rows: {len(train)}")

train = train[~train['clean_source'].isin(discrepancy_df['clean_source'])]

print(f"Number of rows after dropping discrepancies: {len(train)}")

Original number of rows: 376274
Number of rows after dropping discrepancies: 376267


In [31]:
train[['clean_source','masked_source']]

,clean_source,masked_source
0,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...
1,عزيزي جادون، إن بحثك حول دول المراقبة وانتهاكا...,عزيزي [MASK]، إن بحثك حول دول المراقبة وانتهاك...
2,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...
3,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...
4,في 29067 جزر راث، شيد مؤخرا مركز جديد للتربية ...,في [MASK] [MASK] [MASK]، شيد مؤخرا مركز جديد ل...
...,...,...
376269,تم تسجيل أشرف في معسكر STEM. تم تحصيل الرسوم م...,تم تسجيل [MASK] في معسكر STEM. تم تحصيل الرسوم...
376270,بالنسبة للمسابقة العلمية، دور على IPV6 5bf9:9d...,بالنسبة للمسابقة العلمية، دور على IPV6 [MASK][...
376271,انتبه جاكي ريبين، يمكن العثور على معلومات بشأن...,انتبه [MASK] [MASK]، يمكن العثور على معلومات ب...
376272,جمعنا بيانات من جبل طابور. للوصول إلى بيانات إ...,جمعنا بيانات من جبل طابور. للوصول إلى بيانات إ...


In [32]:
train_final = train[['clean_source', 'masked_source']].copy()
train_final.rename(columns={'clean_source': 'source', 'masked_source': 'target'}, inplace = True)

train_final.head()

# Save the train data after the updates
train_final.to_csv('/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv', index=False)

# Loading Model

In [1]:
import pandas as pd
from google.colab import drive
import random

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv")

## Gemma 3 (4B)

In [4]:
!pip install -qU unsloth
from unsloth import FastLanguageModel
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 682.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [37]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = False, # LoRA 16-bit
    dtype = torch.float16,
    full_finetuning = False
)

==((====))==  Unsloth 2025.6.2: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
dfsdgdfgdfg

## SILMA

### SILMA-2B-4bit

In [ ]:
# !pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
model_id = "silma-ai/SILMA-Kashif-2B-Instruct-v1.0-bnb-4bit" # Quantized version
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = quantization_config,
)

In [ ]:
messages = [
    {"role": "user", "content": "كم عدد أيام الأسبوع؟"},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

input_ids

In [ ]:
outputs = model.generate(**input_ids, max_new_tokens=256)

In [ ]:
tokenizer.decode(outputs[0]).split("<start_of_turn>")[-1]

### SILMA-2B

In [ ]:
import torch
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model="silma-ai/SILMA-Kashif-2B-Instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda"
)

In [ ]:
messages = [
    {"role": "user",
     "content": """ Mask the PII as [masked] in the given source sentence

     source: I sent my email address M.helmy@nu.edu.eg to my supervisor dr. gahda khoriba in Nile University.

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
response

In [ ]:
messages = [
    {"role": "user",
     "content": f""" Mask the PII as [masked] in the given source sentence

     source: {df["source"][0]}

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
print(df["source"][0])
print(response)